In [1]:
# Cell 1: Install required packages (safe to re-run)
%pip -q install requests pymongo python-dotenv

import os
import time
import requests
from datetime import datetime
from pymongo import MongoClient, UpdateOne

print("Packages imported successfully.")



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: C:\Users\engr_\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Packages imported successfully.


In [2]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

env_path = os.path.join(os.getcwd(), ".env")
print("Looking for .env at:", env_path)

load_dotenv(env_path)

API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
MONGO_URI = os.getenv("MONGODB_URI")

print("API_KEY loaded:", API_KEY is not None)
print("MONGO_URI loaded:", MONGO_URI is not None)

# Print a safe preview (no secrets) to confirm format
if MONGO_URI:
    print("MONGO_URI preview:", MONGO_URI.split("@")[-1][:40] + "...")

assert API_KEY is not None, "ALPHA_VANTAGE_API_KEY not found in .env"
assert MONGO_URI is not None, "MONGODB_URI not found in .env"

client = MongoClient(MONGO_URI)
db = client["stocks"]
collection = db["raw_prices"]

print("Estimated docs in raw_prices:", collection.estimated_document_count())
print("Connected to MongoDB successfully.")


Looking for .env at: C:\Users\engr_\Stocks-Analysis\.env
API_KEY loaded: True
MONGO_URI loaded: True
MONGO_URI preview: cluster0.kkkaco7.mongodb.net/?appName=Cl...
Estimated docs in raw_prices: 0
Connected to MongoDB successfully.


In [4]:
import requests

def fetch_daily_free(symbol: str, api_key: str) -> dict:
    url = (
        "https://www.alphavantage.co/query"
        f"?function=TIME_SERIES_DAILY"
        f"&symbol={symbol}"
        f"&outputsize=compact"
        f"&apikey={api_key}"
    )
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

payload = fetch_daily_free("AAPL", API_KEY)

if "Error Message" in payload:
    print("ERROR:", payload["Error Message"])
elif "Note" in payload:
    print("RATE LIMIT NOTE:", payload["Note"])
elif "Information" in payload:
    print("INFO:", payload["Information"])
else:
    meta = payload.get("Meta Data", {})
    ts = payload.get("Time Series (Daily)", {})
    print("Meta keys:", list(meta.keys()))
    print("Daily rows returned:", len(ts))
    if ts:
        first_date = next(iter(ts.keys()))
        print("Sample date:", first_date, "Sample close:", ts[first_date].get("4. close"))


Meta keys: ['1. Information', '2. Symbol', '3. Last Refreshed', '4. Output Size', '5. Time Zone']
Daily rows returned: 100
Sample date: 2025-12-26 Sample close: 273.4000


In [6]:
import requests
from datetime import datetime, timezone

def fetch_global_quote(symbol: str, api_key: str) -> dict:
    url = (
        "https://www.alphavantage.co/query"
        f"?function=GLOBAL_QUOTE"
        f"&symbol={symbol}"
        f"&apikey={api_key}"
    )
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

q = fetch_global_quote("AAPL", API_KEY)

if "Error Message" in q:
    print("ERROR:", q["Error Message"])
elif "Note" in q:
    print("RATE LIMIT NOTE:", q["Note"])
elif "Information" in q:
    print("INFO:", q["Information"])
else:
    quote = q.get("Global Quote", {})
    print("Keys:", list(quote.keys()))
    print("Symbol:", quote.get("01. symbol"))
    print("Price:", quote.get("05. price"))
    print("Volume:", quote.get("06. volume"))
    print("Latest trading day:", quote.get("07. latest trading day"))
    print("Pulled at (UTC):", datetime.now(timezone.utc).isoformat())


Keys: ['01. symbol', '02. open', '03. high', '04. low', '05. price', '06. volume', '07. latest trading day', '08. previous close', '09. change', '10. change percent']
Symbol: AAPL
Price: 273.4000
Volume: 21521802
Latest trading day: 2025-12-26
Pulled at (UTC): 2025-12-27T12:41:58.812823+00:00


In [8]:
from datetime import datetime, timezone
from pymongo import MongoClient

# Use correct DB name casing
db = client["Stocks"]
collection = db["raw_prices"]

def quote_to_doc(payload: dict, pulled_at_utc: datetime) -> dict:
    q = payload.get("Global Quote", {})
    ts = pulled_at_utc.astimezone(timezone.utc).replace(tzinfo=None)

    def val(k, default=None):
        v = q.get(k)
        return default if v in (None, "") else v

    return {
        "symbol": val("01. symbol"),
        "timestamp": ts,
        "interval": "quote",
        "open": float(val("02. open", 0.0)),
        "high": float(val("03. high", 0.0)),
        "low": float(val("04. low", 0.0)),
        "close": float(val("05. price", 0.0)),
        "volume": int(float(val("06. volume", 0))),
        "latest_trading_day": val("07. latest trading day"),
        "prev_close": float(val("08. previous close", 0.0)),
        "change": float(val("09. change", 0.0)),
        "change_percent": val("10. change percent"),
        "source": "alpha_vantage",
        "ingested_at": datetime.now(timezone.utc).replace(tzinfo=None)
    }

pulled_at = datetime.now(timezone.utc)
doc = quote_to_doc(q, pulled_at)

assert doc["symbol"], "Quote parsing failed: symbol missing"

collection.update_one(
    {"symbol": doc["symbol"], "timestamp": doc["timestamp"], "interval": doc["interval"]},
    {"$set": doc},
    upsert=True
)

print("Stored quote snapshot for", doc["symbol"])
print("raw_prices total docs now:", collection.estimated_document_count())


Stored quote snapshot for AAPL
raw_prices total docs now: 1


In [9]:
import time
import requests
from datetime import datetime, timezone
from pymongo import UpdateOne

SYMBOLS = ["AAPL", "MSFT", "NVDA", "AMZN", "TSLA", "APP", "SMCI", "SPY"]

def fetch_daily_free(symbol: str, api_key: str, outputsize: str = "compact") -> dict:
    url = (
        "https://www.alphavantage.co/query"
        f"?function=TIME_SERIES_DAILY"
        f"&symbol={symbol}"
        f"&outputsize={outputsize}"
        f"&apikey={api_key}"
    )
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

def parse_daily_docs(symbol: str, payload: dict) -> list[dict]:
    ts = payload.get("Time Series (Daily)", {})
    docs = []
    for date_str, v in ts.items():
        docs.append({
            "symbol": symbol,
            "timestamp": datetime.fromisoformat(date_str),  # date only
            "interval": "1day",
            "open": float(v["1. open"]),
            "high": float(v["2. high"]),
            "low": float(v["3. low"]),
            "close": float(v["4. close"]),
            "volume": int(float(v["5. volume"])),
            "source": "alpha_vantage",
            "ingested_at": datetime.now(timezone.utc).replace(tzinfo=None)
        })
    return docs

def upsert_many(docs: list[dict]) -> int:
    if not docs:
        return 0
    ops = [
        UpdateOne(
            {"symbol": d["symbol"], "timestamp": d["timestamp"], "interval": d["interval"]},
            {"$set": d},
            upsert=True
        )
        for d in docs
    ]
    result = collection.bulk_write(ops, ordered=False)
    return (result.upserted_count or 0) + (result.modified_count or 0)

total_written = 0

for sym in SYMBOLS:
    print(f"\n=== DAILY BACKFILL (compact) for {sym} ===")
    payload = fetch_daily_free(sym, API_KEY, outputsize="compact")

    if "Note" in payload:
        print("RATE LIMIT NOTE:", payload["Note"])
        print("Stopping here. Wait 60 seconds and rerun this cell.")
        break
    if "Error Message" in payload:
        print("ERROR:", payload["Error Message"])
        continue
    if "Information" in payload:
        print("INFO:", payload["Information"])
        continue

    docs = parse_daily_docs(sym, payload)
    written = upsert_many(docs)
    total_written += written

    print(f"Rows fetched: {len(docs)}, upserted: {written}")
    time.sleep(15)  # stay under free tier limits

print("\nTOTAL upserted in this run:", total_written)
print("raw_prices total docs now:", collection.estimated_document_count())



=== DAILY BACKFILL (compact) for AAPL ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for MSFT ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for NVDA ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for AMZN ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for TSLA ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for APP ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for SMCI ===
Rows fetched: 100, upserted: 100

=== DAILY BACKFILL (compact) for SPY ===
Rows fetched: 100, upserted: 100

TOTAL upserted in this run: 800
raw_prices total docs now: 801


In [10]:
import time
import requests
from datetime import datetime, date, timedelta, timezone
from pymongo import UpdateOne

SYMBOLS = ["AAPL", "MSFT", "NVDA", "AMZN", "TSLA", "APP", "SMCI", "SPY"]

# Keep last 3 years (approx). You can tighten later if needed.
cutoff_date = date.today() - timedelta(days=365*3)
print("Cutoff date (keep >=):", cutoff_date)

def fetch_daily_free(symbol: str, api_key: str, outputsize: str = "full") -> dict:
    url = (
        "https://www.alphavantage.co/query"
        f"?function=TIME_SERIES_DAILY"
        f"&symbol={symbol}"
        f"&outputsize={outputsize}"
        f"&apikey={api_key}"
    )
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

def parse_daily_docs_3y(symbol: str, payload: dict, cutoff: date) -> list[dict]:
    ts = payload.get("Time Series (Daily)", {})
    docs = []
    for date_str, v in ts.items():
        d = date.fromisoformat(date_str)
        if d < cutoff:
            continue
        docs.append({
            "symbol": symbol,
            "timestamp": datetime.fromisoformat(date_str),
            "interval": "1day",
            "open": float(v["1. open"]),
            "high": float(v["2. high"]),
            "low": float(v["3. low"]),
            "close": float(v["4. close"]),
            "volume": int(float(v["5. volume"])),
            "source": "alpha_vantage",
            "ingested_at": datetime.now(timezone.utc).replace(tzinfo=None)
        })
    return docs

def upsert_many(docs: list[dict]) -> int:
    if not docs:
        return 0
    ops = [
        UpdateOne(
            {"symbol": d["symbol"], "timestamp": d["timestamp"], "interval": d["interval"]},
            {"$set": d},
            upsert=True
        )
        for d in docs
    ]
    result = collection.bulk_write(ops, ordered=False)
    return (result.upserted_count or 0) + (result.modified_count or 0)

total_written = 0

for sym in SYMBOLS:
    print(f"\n=== DAILY BACKFILL (full, last 3y) for {sym} ===")
    payload = fetch_daily_free(sym, API_KEY, outputsize="full")

    if "Note" in payload:
        print("RATE LIMIT NOTE:", payload["Note"])
        print("Stop here. Wait 60 seconds and rerun this cell.")
        break
    if "Error Message" in payload:
        print("ERROR:", payload["Error Message"])
        continue
    if "Information" in payload:
        print("INFO:", payload["Information"])
        continue

    docs = parse_daily_docs_3y(sym, payload, cutoff_date)
    written = upsert_many(docs)
    total_written += written

    print(f"Rows kept (>= cutoff): {len(docs)}, upserted: {written}")
    time.sleep(15)

print("\nTOTAL upserted in this run:", total_written)
print("raw_prices total docs now:", collection.estimated_document_count())


Cutoff date (keep >=): 2022-12-28

=== DAILY BACKFILL (full, last 3y) for AAPL ===
INFO: Thank you for using Alpha Vantage! The outputsize=full parameter value is a premium feature for the TIME_SERIES_DAILY endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium features

=== DAILY BACKFILL (full, last 3y) for MSFT ===
INFO: Thank you for using Alpha Vantage! The outputsize=full parameter value is a premium feature for the TIME_SERIES_DAILY endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium features

=== DAILY BACKFILL (full, last 3y) for NVDA ===
INFO: Thank you for using Alpha Vantage! The outputsize=full parameter value is a premium feature for the TIME_SERIES_DAILY endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium features

=== DAILY BACKFILL (full, last 3y) 

In [11]:
import csv
import io
import requests
from datetime import datetime, date, timedelta, timezone
from pymongo import UpdateOne

cutoff_date = date.today() - timedelta(days=365*3)
print("Cutoff date (keep >=):", cutoff_date)

def stooq_symbol(sym: str) -> str:
    # Stooq uses lower-case and ".us" for US equities/ETFs
    return sym.lower() + ".us"

def fetch_stooq_daily_csv(sym: str) -> str:
    s = stooq_symbol(sym)
    url = f"https://stooq.com/q/d/l/?s={s}&i=d"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.text

def parse_stooq_csv(sym: str, csv_text: str, cutoff: date) -> list[dict]:
    reader = csv.DictReader(io.StringIO(csv_text))
    docs = []
    for row in reader:
        # Row keys: Date, Open, High, Low, Close, Volume
        d = date.fromisoformat(row["Date"])
        if d < cutoff:
            continue
        docs.append({
            "symbol": sym,
            "timestamp": datetime.fromisoformat(row["Date"]),
            "interval": "1day",
            "open": float(row["Open"]),
            "high": float(row["High"]),
            "low": float(row["Low"]),
            "close": float(row["Close"]),
            "volume": int(float(row["Volume"])),
            "source": "stooq",
            "ingested_at": datetime.now(timezone.utc).replace(tzinfo=None)
        })
    return docs

def upsert_many(docs: list[dict]) -> int:
    if not docs:
        return 0
    ops = [
        UpdateOne(
            {"symbol": d["symbol"], "timestamp": d["timestamp"], "interval": d["interval"]},
            {"$set": d},
            upsert=True
        )
        for d in docs
    ]
    result = collection.bulk_write(ops, ordered=False)
    return (result.upserted_count or 0) + (result.modified_count or 0)

# Test with ONE symbol first
sym = "AAPL"
csv_text = fetch_stooq_daily_csv(sym)
docs = parse_stooq_csv(sym, csv_text, cutoff_date)

print("Rows parsed (>= cutoff):", len(docs))
written = upsert_many(docs)
print("Upserted:", written)
print("raw_prices total docs now:", collection.estimated_document_count())


Cutoff date (keep >=): 2022-12-28
Rows parsed (>= cutoff): 752
Upserted: 752
raw_prices total docs now: 1453


In [12]:
import csv
import io
import requests
from datetime import datetime, date, timedelta, timezone
from pymongo import UpdateOne

SYMBOLS = ["AAPL", "MSFT", "NVDA", "AMZN", "TSLA", "APP", "SMCI", "SPY"]

cutoff_date = date.today() - timedelta(days=365*3)
print("Cutoff date (keep >=):", cutoff_date)

def stooq_symbol(sym: str) -> str:
    return sym.lower() + ".us"

def fetch_stooq_daily_csv(sym: str) -> str:
    url = f"https://stooq.com/q/d/l/?s={stooq_symbol(sym)}&i=d"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.text

def parse_stooq_csv(sym: str, csv_text: str, cutoff: date) -> list[dict]:
    reader = csv.DictReader(io.StringIO(csv_text))
    docs = []
    for row in reader:
        d = date.fromisoformat(row["Date"])
        if d < cutoff:
            continue
        docs.append({
            "symbol": sym,
            "timestamp": datetime.fromisoformat(row["Date"]),
            "interval": "1day",
            "open": float(row["Open"]),
            "high": float(row["High"]),
            "low": float(row["Low"]),
            "close": float(row["Close"]),
            "volume": int(float(row["Volume"])),
            "source": "stooq",
            "ingested_at": datetime.now(timezone.utc).replace(tzinfo=None)
        })
    return docs

def upsert_many(docs: list[dict]) -> int:
    if not docs:
        return 0
    ops = [
        UpdateOne(
            {"symbol": d["symbol"], "timestamp": d["timestamp"], "interval": d["interval"]},
            {"$set": d},
            upsert=True
        )
        for d in docs
    ]
    result = collection.bulk_write(ops, ordered=False)
    return (result.upserted_count or 0) + (result.modified_count or 0)

total_upserted = 0

for sym in SYMBOLS:
    print(f"\n=== STOOQ DAILY (last 3y) for {sym} ===")
    try:
        csv_text = fetch_stooq_daily_csv(sym)
        docs = parse_stooq_csv(sym, csv_text, cutoff_date)
        print("Rows parsed:", len(docs))
        written = upsert_many(docs)
        total_upserted += written
        print("Upserted:", written)
    except Exception as e:
        print("FAILED:", sym, "|", repr(e))

print("\nTOTAL upserted this run:", total_upserted)
print("raw_prices total docs now:", collection.estimated_document_count())


Cutoff date (keep >=): 2022-12-28

=== STOOQ DAILY (last 3y) for AAPL ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for MSFT ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for NVDA ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for AMZN ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for TSLA ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for APP ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for SMCI ===
Rows parsed: 752
Upserted: 752

=== STOOQ DAILY (last 3y) for SPY ===
Rows parsed: 752
Upserted: 752

TOTAL upserted this run: 6016
raw_prices total docs now: 6017


In [13]:
import time
import requests
from datetime import datetime, timezone

SYMBOLS = ["AAPL", "MSFT", "NVDA", "AMZN", "TSLA", "APP", "SMCI", "SPY"]
POLL_SECONDS = 300  # 5 minutes

def fetch_global_quote(symbol: str, api_key: str) -> dict:
    url = (
        "https://www.alphavantage.co/query"
        f"?function=GLOBAL_QUOTE"
        f"&symbol={symbol}"
        f"&apikey={api_key}"
    )
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

def quote_to_doc(payload: dict, pulled_at_utc: datetime) -> dict:
    q = payload.get("Global Quote", {})
    ts = pulled_at_utc.astimezone(timezone.utc).replace(tzinfo=None)

    def val(k, default=None):
        v = q.get(k)
        return default if v in (None, "") else v

    return {
        "symbol": val("01. symbol"),
        "timestamp": ts,
        "interval": "quote",
        "open": float(val("02. open", 0.0)),
        "high": float(val("03. high", 0.0)),
        "low": float(val("04. low", 0.0)),
        "close": float(val("05. price", 0.0)),
        "volume": int(float(val("06. volume", 0))),
        "latest_trading_day": val("07. latest trading day"),
        "prev_close": float(val("08. previous close", 0.0)),
        "change": float(val("09. change", 0.0)),
        "change_percent": val("10. change percent"),
        "source": "alpha_vantage",
        "ingested_at": datetime.now(timezone.utc).replace(tzinfo=None)
    }

def store_quote_snapshot(symbol: str):
    pulled_at = datetime.now(timezone.utc)
    payload = fetch_global_quote(symbol, API_KEY)

    if "Note" in payload:
        print("RATE LIMIT NOTE:", payload["Note"])
        return False
    if "Error Message" in payload:
        print("ERROR:", symbol, payload["Error Message"])
        return False
    if "Information" in payload:
        print("INFO:", symbol, payload["Information"])
        return False

    doc = quote_to_doc(payload, pulled_at)
    if not doc.get("symbol"):
        print("Parse failed for", symbol)
        return False

    collection.update_one(
        {"symbol": doc["symbol"], "timestamp": doc["timestamp"], "interval": doc["interval"]},
        {"$set": doc},
        upsert=True
    )
    print("Stored quote:", symbol, "| ts:", doc["timestamp"])
    return True

print("Starting quote polling loop. Stop with Kernel -> Interrupt.")
while True:
    start = datetime.now(timezone.utc)
    ok = 0
    for sym in SYMBOLS:
        if store_quote_snapshot(sym):
            ok += 1
        time.sleep(15)  # spacing calls to reduce throttling risk

    print(f"Cycle complete: {ok}/{len(SYMBOLS)} stored. UTC:", start.isoformat())
    print(f"Sleeping {POLL_SECONDS} seconds...\n")
    time.sleep(POLL_SECONDS)


Starting quote polling loop. Stop with Kernel -> Interrupt.
Stored quote: AAPL | ts: 2025-12-27 13:43:11.990559
Stored quote: MSFT | ts: 2025-12-27 13:43:28.591844
Stored quote: NVDA | ts: 2025-12-27 13:43:44.879372
Stored quote: AMZN | ts: 2025-12-27 13:44:01.259338
Stored quote: TSLA | ts: 2025-12-27 13:44:17.750039
Stored quote: APP | ts: 2025-12-27 13:44:34.335342
Stored quote: SMCI | ts: 2025-12-27 13:44:51.027867
Stored quote: SPY | ts: 2025-12-27 13:45:07.308955
Cycle complete: 8/8 stored. UTC: 2025-12-27T13:43:11.990559+00:00
Sleeping 300 seconds...



KeyboardInterrupt: 